In [5]:
import pandas as pd
import datetime as dt
import sqlite3 as db

### Importando os dados e concatenando tudo em um único data frame

In [19]:
dados1 = pd.read_csv("/home/abel/Documentos/Github/Pessoal/GitHub/HelpDeskSearch/data/helpdesk.csv", sep=';')
dados2 = pd.read_csv("/home/abel/Documentos/Github/Pessoal/GitHub/HelpDeskSearch/data/helpdesk2.csv", sep=',')
dados3 = pd.read_csv("/home/abel/Documentos/Github/Pessoal/GitHub/HelpDeskSearch/data/helpdesk3.csv", sep=';', encoding='latin1')
dados4 = pd.read_csv("/home/abel/Documentos/Github/Pessoal/GitHub/HelpDeskSearch/data/helpdesk4.csv", sep=';',encoding='latin1')
#Alterando os nome das colunas para realizar o agrupamentos das bases de forma correta
#dados 3
dados3.columns.values[5] = "Data Solução"
dados3.columns.values[8] = "Satisfação"
dados3.columns.values[9] = "Data Iteração"   
dados3.columns.values[10] = "Usuário Iteração"   
#concatenando
dados_agrupados = pd.concat([dados1,dados2,dados3,dados4])

### Visualizando os dados

In [12]:
dados_agrupados.shape

(58926, 12)

In [ ]:
dados_agrupados.head()

In [20]:
dados_agrupados['Categoria'].str.replace('-',"Sem Categoria Definida")
dados_agrupados['Resolvido'] = dados_agrupados['Resolvido'].str.replace("nÆo","não")
#Classificando as satisfações
dados_agrupados['Satisfação'] = dados_agrupados['Satisfação'].str.replace("-","Sem Nota")
dados_agrupados['Satisfação'] = dados_agrupados['Satisfação'].str.replace("0","Detrator")
dados_agrupados['Satisfação'] = dados_agrupados['Satisfação'].str.replace("1","Detrator")
dados_agrupados['Satisfação'] = dados_agrupados['Satisfação'].str.replace("2","Detrator")
dados_agrupados['Satisfação'] = dados_agrupados['Satisfação'].str.replace("3","Neutro")
dados_agrupados['Satisfação'] = dados_agrupados['Satisfação'].str.replace("4","Promotor")
dados_agrupados['Satisfação'] = dados_agrupados['Satisfação'].str.replace("5","Promotor")
#transformando para formato data as colunas do tipo data ou tempo
dados_agrupados['Data Fechamento'] = dados_agrupados['Data Fechamento'].str.replace("-",'')
dados_agrupados['Data Solução']= dados_agrupados['Data Solução'].str.replace("-",'')
dados_agrupados['Data Iteração']= dados_agrupados['Data Iteração'].str.replace("-",'')

dados_agrupados[["Data Abertura", "Data Fechamento", "Data Solução", "Data Iteração"]] = dados_agrupados[["Data Abertura", "Data Fechamento", "Data Solução", "Data Iteração"]].apply(pd.to_datetime)
#criando novas features
dados_agrupados['tempo_gasto_solucao'] = dados_agrupados['Data Solução'] - dados_agrupados['Data Abertura']
dados_agrupados['tempo_gasto_fechamento'] = dados_agrupados['Data Fechamento'] - dados_agrupados['Data Abertura']

In [25]:
conn = db.connect('help_desk_database')
c = conn.cursor()
c.execute(''' CREATE TABLE IF NOT EXISTS chamados (Ticker number)''')
conn.commit()
dados_agrupados.to_sql('chamados',conn,if_exists='replace',index=False)


/tmp/ipykernel_293128/3553859856.py:5: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  dados_agrupados.to_sql('chamados',conn,if_exists='replace',index=False)


58926

In [26]:
pd.read_sql(f'''SELECT * FROM chamados
                            WHERE Categoria like "%SIPAC%"
                            AND "Data Abertura" >= "2020-01-01"
                            AND "Data Abertura" <= "2022-01-01"
                             ''',conn)

,Ticker,Requerente,Grupo,Categoria,Data Abertura,Data Solução,Data Fechamento,Atendente,Satisfação,Data Iteração,Usuário Iteração,Resolvido,tempo_gasto_solucao,tempo_gasto_fechamento
0,36937,ONIVALDO FLORES JUNIOR,SIPAC,PROSSIGA > SIPAC > PROTOCOLO,2020-07-21 09:16:13,2020-06-08 17:15:33,2020-08-08 01:16:14,GUSTAVO COSTA BRAUNER,Sem Nota,2020-07-21 13:02:43,GUSTAVO COSTA BRAUNER,sim,-3686440000000000,1526401000000000
1,36937,ONIVALDO FLORES JUNIOR,SIPAC,PROSSIGA > SIPAC > PROTOCOLO,2020-07-21 09:16:13,2020-06-08 17:15:33,2020-08-08 01:16:14,GUSTAVO COSTA BRAUNER,Sem Nota,2020-07-21 13:19:54,GUSTAVO COSTA BRAUNER,sim,-3686440000000000,1526401000000000
2,36937,ONIVALDO FLORES JUNIOR,SIPAC,PROSSIGA > SIPAC > PROTOCOLO,2020-07-21 09:16:13,2020-06-08 17:15:33,2020-08-08 01:16:14,GUSTAVO COSTA BRAUNER,Sem Nota,2020-05-08 11:54:49,ONIVALDO FLORES JUNIOR,sim,-3686440000000000,1526401000000000
3,36937,ONIVALDO FLORES JUNIOR,SIPAC,PROSSIGA > SIPAC > PROTOCOLO,2020-07-21 09:16:13,2020-06-08 17:15:33,2020-08-08 01:16:14,GUSTAVO COSTA BRAUNER,Sem Nota,2020-05-08 15:46:58,GUSTAVO COSTA BRAUNER,sim,-3686440000000000,1526401000000000
4,36937,ONIVALDO FLORES JUNIOR,SIPAC,PROSSIGA > SIPAC > PROTOCOLO,2020-07-21 09:16:13,2020-06-08 17:15:33,2020-08-08 01:16:14,GUSTAVO COSTA BRAUNER,Sem Nota,2020-06-08 15:45:25,ONIVALDO FLORES JUNIOR,sim,-3686440000000000,1526401000000000
5,37211,ONIVALDO FLORES JUNIOR,Banco de Dados,PROSSIGA > SIPAC > PROTOCOLO,2020-08-25 16:20:18,2020-03-09 07:56:44,2020-04-09 11:39:56,MARCO AURELIO ROCHA,Sem Nota,2020-08-25 18:29:51,MARCO AURELIO ROCHA,sim,-14631814000000000,-11940022000000000
6,37211,ONIVALDO FLORES JUNIOR,Problemas GLPI,PROSSIGA > SIPAC > PROTOCOLO,2020-08-25 16:20:18,2020-03-09 07:56:44,2020-04-09 11:39:56,MARCO AURELIO ROCHA,Sem Nota,2020-08-25 18:29:51,MARCO AURELIO ROCHA,sim,-14631814000000000,-11940022000000000
7,37211,ONIVALDO FLORES JUNIOR,PROTOCOLO - NEGOCIAL,PROSSIGA > SIPAC > PROTOCOLO,2020-08-25 16:20:18,2020-03-09 07:56:44,2020-04-09 11:39:56,MARCO AURELIO ROCHA,Sem Nota,2020-08-25 18:29:51,MARCO AURELIO ROCHA,sim,-14631814000000000,-11940022000000000
8,37211,ONIVALDO FLORES JUNIOR,SEI,PROSSIGA > SIPAC > PROTOCOLO,2020-08-25 16:20:18,2020-03-09 07:56:44,2020-04-09 11:39:56,MARCO AURELIO ROCHA,Sem Nota,2020-08-25 18:29:51,MARCO AURELIO ROCHA,sim,-14631814000000000,-11940022000000000
9,37211,ONIVALDO FLORES JUNIOR,SIPAC,PROSSIGA > SIPAC > PROTOCOLO,2020-08-25 16:20:18,2020-03-09 07:56:44,2020-04-09 11:39:56,MARCO AURELIO ROCHA,Sem Nota,2020-08-25 18:29:51,MARCO AURELIO ROCHA,sim,-14631814000000000,-11940022000000000


In [25]:
dados_agrupados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58926 entries, 0 to 3567
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   Ticker                  58926 non-null  int64          
 1   Requerente              58926 non-null  object         
 2   Grupo                   58926 non-null  object         
 3   Categoria               58926 non-null  object         
 4   Data Abertura           58926 non-null  datetime64[ns] 
 5   Data Solução            54484 non-null  datetime64[ns] 
 6   Data Fechamento         54173 non-null  datetime64[ns] 
 7   Atendente               58926 non-null  object         
 8   Satisfação              58926 non-null  object         
 9   Data Iteração           39876 non-null  datetime64[ns] 
 10  Usuário Iteração        58926 non-null  object         
 11  Resolvido               58926 non-null  object         
 12  tempo_gasto_solucao     54484 non

In [21]:
#Exportando
dados_agrupados.to_csv("data/Helpdesk_consolidado.csv", index=False)

In [156]:
#Carregando os com as datas tratadas
dados = pd.read_csv("/home/abel/Documentos/Github/Pessoal/GitHub/HelpDeskSearch/Helpdesk_consolidado.csv")

## Convertendo dados de data do tipo object para datetime

In [157]:
dados[["Data Abertura", "Data Fechamento", "Data Solução", "Data Iteração"]] = dados[["Data Abertura", "Data Fechamento", "Data Solução", "Data Iteração"]].apply(pd.to_datetime)

In [ ]:
dados.info()

### Criação de features (Novas Colunas)

In [159]:
dados['tempo_gasto_solucao'] = dados['Data Solução'] - dados['Data Abertura']
dados['tempo_gasto_fechamento'] = dados['Data Fechamento'] - dados['Data Abertura']

In [160]:
#Removendo colunas que não serão necessárias
dados.drop(columns=['Requerente', 'Atendente','Usuário Iteração'], inplace=True)

In [164]:
#Exportando a base final
dados.to_csv("Helpdesk_final.csv", index=False)